In [2]:
import numpy as np
from keras.applications import VGG16

size = (224, 224, 3)

vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=size)

for layer in vgg16.layers:
    layer.trainable = False

In [3]:
for i, layer in enumerate(vgg16.layers):
    print(str(i)+" "+layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [4]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

num_classes =2

l = vgg16.output
l = Flatten(name='flatten')(l)
fc1 = Dense(256, activation='relu')(l)
fc1 = Dropout(0.3)(fc1)
fc_head = Dense(num_classes, activation='softmax')(fc1)

model = Model(inputs = vgg16.input, outputs=fc_head)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = './train'

train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(224, 224),
                                                   batch_size=6,
                                                   class_mode='categorical')

Found 50 images belonging to 2 classes.


In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

n_train = 50
# n_validation = 246
epochs = 15
batch_size = 6

model.compile(loss='categorical_crossentropy',
               optimizer=Adam(lr=0.001), metrics=['accuracy'])

hostory = model.fit_generator(train_generator,
                             steps_per_epoch=n_train//batch_size,
                              epochs=epochs)
#                              callbacks = callbacks,
#                              validation_data = validation_generator,
#                              validation_steps = n_validation//batch_size)

model.save('save.h5')

In [9]:
from keras.preprocessing import image
import cv2
e1=0
e2=0
for i in range(1, 20):
    pred = image.load_img('./test/'+str(i)+'.jpg', target_size=(224, 224))
    img = cv2.imread('./test/'+str(i)+'.jpg')
    pred = image.img_to_array(pred)
    pred = np.expand_dims(pred, axis=0)

    result = model.predict(pred)
    if np.argmax(result[0]==0):
        print("Healthy")
        cv2.imshow("Healthy", img)
        cv2.waitKey()
        cv2.destroyAllWindows()
        e1+=1
    else:
        print("Rice blast!", i)
        cv2.imshow("Rice blast", img)
        cv2.waitKey()
        cv2.destroyAllWindows()
        e2+=1

Rice blast! 1
Rice blast! 2
Rice blast! 3
Rice blast! 4
Rice blast! 5
Rice blast! 6
Healthy
Rice blast! 8
Rice blast! 9
Rice blast! 10
Rice blast! 11
Rice blast! 12
Rice blast! 13
Rice blast! 14
Rice blast! 15
Rice blast! 16
Healthy
Rice blast! 18
Rice blast! 19
